<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Data Analytics 

### Interactive Parametric Distributions  in Python 


#### Michael Pyrcz, Associate Professor, University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)


### Data Analytics: Parametric Distributions

Here's a demonstration of making and general use of parametric distributions in Python. This demonstration is part of the resources that I include for my courses in Spatial / Subsurface Data Analytics at the Cockrell School of Engineering at the University of Texas at Austin.  

#### Parametric Distributions

I provide an set of interactives with thses parametric distributions:

* Uniform
* Triangular
* Binomial
* Poisson
* Gaussian
* Log Normal

Change the distribution parameters and watch the probability density functions and cumulative distribution functions changes.

I have a lecture on these parametric distributions available on [YouTube](https://www.youtube.com/watch?v=U7fGsqCLPHU&t=1687s).   

#### Getting Started

Here's the steps to get setup in Python with the GeostatsPy package:

1. Install Anaconda 3 on your machine (https://www.anaconda.com/download/). 
2. From Anaconda Navigator (within Anaconda3 group), go to the environment tab, click on base (root) green arrow and open a terminal. 
3. In the terminal type: pip install geostatspy. 
4. Open Jupyter and in the top block get started by copy and pasting the code block below from this Jupyter Notebook to start using the geostatspy functionality. 

#### Importing Packages

We will need some standard packages. These should have been installed with Anaconda 3.

In [3]:
import numpy as np                        # ndarrys for gridded data
import pandas as pd                       # DataFrames for tabular data
import os                                 # set working directory, run executables
import matplotlib.pyplot as plt           # for plotting
from scipy import stats                   # summary statistics
import math                               # trigonometry etc.
import scipy.signal as signal             # kernel for moving window calculation
import random                             # for randon numbers
import seaborn as sns                     # for matrix scatter plots
from scipy import linalg                  # for linear regression
from sklearn import preprocessing
import geostatspy.GSLIB as GSLIB
from ipywidgets import interactive        # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox

#### Set the Working Directory

I always like to do this so I don't lose files and to simplify subsequent read and writes (avoid including the full address each time). 

#### Uniform Distribution

Let's start with the most simple distribution.

* Maximum uncertainty distribution. Used when very little information available.

In [101]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                                      Uniform Parametric Distribution Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

zmin = widgets.FloatSlider(min=0.0, max = 100.0, value = 10.0, description = 'Min',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
zmin.style.handle_color = 'red'
zmax = widgets.FloatSlider(min=0.0, max = 100.0, value = 30.0, description = 'Max',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
zmax.style.handle_color = 'red'

ui1 = widgets.HBox([zmin,zmax],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})

def f_make(zmin,zmax):
    xvals = np.linspace(0.0,100.0,1000)
    pdf = stats.uniform.pdf(xvals, loc = zmin, scale = zmax-zmin)
    cdf = stats.uniform.cdf(xvals, loc = zmin, scale = zmax-zmin)

    plt.subplot(121)
    plt.plot(xvals,pdf,'--',color='red')
    plt.fill_between(xvals, 0, pdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot([0,100],[0,0],color='black')
    plt.xlim(0,100); plt.gca().set_ylim(bottom=0.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Probability Density Function")
    
    plt.subplot(122)
    plt.fill_between(xvals, 0, cdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot(xvals,cdf,'--',color='red')
    plt.plot([0,100],[0,0],color='black')
    plt.xlim(0,100); plt.ylim(0,1.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Cumulative Distribution Function")
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.3, top=1.2, wspace=0.2, hspace=0.3)  
    plt.plot()

interactive_plot = widgets.interactive_output(f_make, {'zmin':zmin,'zmax':zmax})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

#### Binomial Distribution

Now the binomial distribution.

* a discrete distribution of the probablity of a specific number of successes given $P(success)$ and a number of trails, $n$.

In [84]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                                      Binomial Parametric Distribution Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

zp = widgets.FloatSlider(min=0.01, max = 0.99, value = 0.5, step = 0.01, description = '$P(Success)$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
zp.style.handle_color = 'red'
zn = widgets.IntSlider(min=2, max = 100, value = 10, description = '$N_{trials}$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
zn.style.handle_color = 'red'

ui1 = widgets.HBox([zp,zn],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})

def f_make(zp,zn):
    xvals = np.linspace(1,zn,zn)
    pdf = stats.binom.pmf(xvals, zn, zp)
    cdf = stats.binom.cdf(xvals, zn, zp)
    
    plt.subplot(121)
    plt.plot(xvals,pdf,'--',color='red')
    plt.fill_between(xvals, 0, pdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot([1,zn],[0,0],color='black')
    plt.xlim(1,zn); plt.gca().set_ylim(bottom=0.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Probability Density Function")
    
    plt.subplot(122)
    plt.fill_between(xvals, 0, cdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot(xvals,cdf,'--',color='red')
    plt.plot([0,zn],[0,0],color='black')
    plt.xlim(1,zn); plt.ylim(0,1.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Cumulative Distribution Function")
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.3, top=1.2, wspace=0.2, hspace=0.3)  
    plt.plot()

interactive_plot = widgets.interactive_output(f_make, {'zp':zp,'zn':zn})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

#### Poisson Distribution

Now the Poisson distribution.

* a discrete distribution of the probablity of a specific number of successes given the average number of successes, $\lambda$, over an interval.

In [92]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                                      Poisson Parametric Distribution Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

zlambda = widgets.IntSlider(min=1, max = 15, value = 5, step = 1.0, description = '$\\lambda$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
zlambda.style.handle_color = 'red'

ui1 = widgets.HBox([zlambda],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})

def f_make(zlambda):
    n = 30
    xvals = np.linspace(1,n,n)
    pdf = stats.poisson.pmf(xvals, zlambda)
    cdf = stats.poisson.cdf(xvals, zlambda)
    
    plt.subplot(121)
    plt.plot(xvals,pdf,'--',color='red')
    plt.fill_between(xvals, 0, pdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot([1,n],[0,0],color='black')
    plt.xlim(1,n); plt.ylim(0,0.40)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Probability Density Function")
    
    plt.subplot(122)
    plt.fill_between(xvals, 0, cdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot(xvals,cdf,'--',color='red')
    plt.plot([0,n],[0,0],color='black')
    plt.xlim(1,n); plt.ylim(0,1.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Cumulative Distribution Function")
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.3, top=1.2, wspace=0.2, hspace=0.3)  
    plt.plot()

interactive_plot = widgets.interactive_output(f_make, {'zlambda':zlambda})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

#### Gaussian / Normal Distribution

Now the Gaussian distribution.

* an unbounded distribution (infinite tails) distribution parameterized by mean and standard deviation.

In [99]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                                      Gaussian Parametric Distribution Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

zmean = widgets.FloatSlider(min=0.0, max = 100.0, value = 50.0, description = 'mean/$\\mu$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
zmean.style.handle_color = 'red'
zstdev = widgets.FloatSlider(min=0.0, max = 30.0, value = 5.0, description = '$\\sigma$',orientation='horizontal',layout=Layout(width='400px', height='50px'),continuous_update=False)
zstdev.style.handle_color = 'red'

ui1 = widgets.HBox([zmean,zstdev],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})

def f_make(zmean,zstdev):
    xvals = np.linspace(0.0,100.0,1000)
    pdf = stats.norm.pdf(xvals, loc = zmean, scale = zstdev)
    cdf = stats.norm.cdf(xvals, loc = zmean, scale = zstdev)

    plt.subplot(121)
    plt.plot(xvals,pdf,'--',color='red')
    plt.fill_between(xvals, 0, pdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot([0,100],[0,0],color='black')
    plt.xlim(0,100); plt.gca().set_ylim(bottom=0.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Probability Density Function")
    
    plt.subplot(122)
    plt.fill_between(xvals, 0, cdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot(xvals,cdf,'--',color='red')
    plt.plot([0,100],[0,0],color='black')
    plt.xlim(0,100); plt.ylim(0,1.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Cumulative Distribution Function")
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.3, top=1.2, wspace=0.2, hspace=0.3)  
    plt.plot()

interactive_plot = widgets.interactive_output(f_make, {'zmean':zmean,'zstdev':zstdev})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

#### Lognormal Distribution

Now the lognormal distribution.

* an unbounded distribution (infinite tails) distribution parameterized by mu and sigma.

In [100]:
# interactive calculation of the random sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='                                      LogNormal Parametric Distribution Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

zmu = widgets.FloatSlider(min=0.1, max = 10.0, value = 1.0, description = 'Mu/$\\mu$',orientation='horizontal',layout=Layout(width='230px', height='50px'),continuous_update=False)
zmu.style.handle_color = 'red'
zsigma = widgets.FloatSlider(min=0.1, max = 3.0, value = 1.0, description = 'Sigma/$\\sigma$',orientation='horizontal',layout=Layout(width='230px', height='50px'),continuous_update=False)
zsigma.style.handle_color = 'red'

ui1 = widgets.HBox([zmu,zsigma],kwargs = {'justify_content':'center'}) 
ui = widgets.VBox([l,ui1],kwargs = {'justify_content':'center'})

def f_make(zmu,zsigma):
    xvals = np.linspace(0.0,100.0,1000)
    pdf = stats.lognorm.pdf(xvals, s = zsigma, scale = math.exp(zmu))
    cdf = stats.lognorm.cdf(xvals, s = zsigma, scale = math.exp(zmu))

    plt.subplot(121)
    plt.plot(xvals,pdf,'--',color='red')
    plt.fill_between(xvals, 0, pdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot([0,100],[0,0],color='black')
    plt.xlim(0,100); plt.gca().set_ylim(bottom=0.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Probability Density Function")
    
    plt.subplot(122)
    plt.fill_between(xvals, 0, cdf, facecolor='red', interpolate=True, alpha = 0.3)
    plt.plot(xvals,cdf,'--',color='red')
    plt.plot([0,100],[0,0],color='black')
    plt.xlim(0,100); plt.ylim(0,1.0)
    plt.xlabel("$X$"); plt.ylabel("Density"); plt.title("Cumulative Distribution Function")
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.3, top=1.2, wspace=0.2, hspace=0.3)  
    plt.plot()

interactive_plot = widgets.interactive_output(f_make, {'zmu':zmu,'zsigma':zsigma})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating    
    
display(ui, interactive_plot)                            # display the interactive plot

Output()

There are many other parametric distributions that we could have included. Also we could have demonstrated the distribution fitting. 

#### Comments

This was a basic interactive demonstration of common parametric distributions. 

I have other demonstrations on the basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations, trend modeling and many other workflows available at [Python Demos](https://github.com/GeostatsGuy/PythonNumericalDemos) and a Python package for data analytics and geostatistics at [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy). 
  
I hope this was helpful,

*Michael*

#### The Author:

### Michael Pyrcz, Associate Professor, University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Associate Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)
